In [1]:
import glob                         # this module helps in selecting files 
import pandas as pd                 # this module helps in processing CSV files
import xml.etree.ElementTree as ET  # this module helps in processing XML files.
from datetime import datetime

In [2]:
#Set paths to files for extraction
tmpfile    = "dealership_temp.tmp"               # file used to store all extracted data
logfile    = "dealership_logfile.txt"            # all event logs will be stored in this file
targetfile = "dealership_transformed_data.csv"   # file where transformed data is stored

In [3]:
#EXTRACT
# Function to extract from CSV files and return a dataframe
def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

# Function to extract from JSON files and return a dataframe
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process,lines=True)
    return dataframe

# Function to extract from XML files and return a dataframe. More complex.
def extract_from_xml(file_to_process):
    dataframe = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel'])
    tree = ET.parse(file_to_process)
    root = tree.getroot()
    for person in root:
        car_model = person.find("car_model").text
        year_of_manufacture = int(person.find("year_of_manufacture").text)
        price = float(person.find("price").text)
        fuel = person.find("fuel").text
        dataframe = dataframe.append({"car_model":car_model, "year_of_manufacture":year_of_manufacture, "price":price, "fuel":fuel}, ignore_index=True)
    return dataframe

# Extract function. Loop through files of each type in directory and compile extracted data.
def extract():
    extracted_data = pd.DataFrame(columns=['car_model','year_of_manufacture','price', 'fuel']) # create an empty data frame to hold extracted data
    
    #process all csv files
    for csvfile in glob.glob("data/dealership_data/*.csv"):
        extracted_data = extracted_data.append(extract_from_csv(csvfile), ignore_index=True)
        
    #process all json files
    for jsonfile in glob.glob("data/dealership_data/*.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    
    #process all xml files
    for xmlfile in glob.glob("data/dealership_data/*.xml"):
        extracted_data = extracted_data.append(extract_from_xml(xmlfile), ignore_index=True)
        
    return extracted_data




In [4]:
#TRANSFORM

# Round price to 2 decimal places.
def transform(data):
        data['price'] = round(data.price, 2)
        return data


In [5]:
# LOAD
def load(targetfile,data_to_load):
    data_to_load.to_csv(targetfile)

In [6]:
# LOG
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [7]:
log("ETL Job Started")

In [13]:
log("Extract phase Started")
extracted_data = extract()
log("Extract phase Ended")
extracted_data

,car_model,year_of_manufacture,price,fuel
0,baleno,2016,8731.343284,Petrol
1,alto k10,2014,3805.970149,Petrol
2,sx4,2008,2910.447761,Petrol
3,dzire,2014,8208.955224,Diesel
4,omni,2012,1865.671642,Petrol
5,ciaz,2014,11194.029851,Petrol
6,ritz,2013,3955.223881,Petrol
7,wagon r,2006,1567.164179,Petrol
8,ertiga,2015,8656.716418,Petrol
9,ciaz,2017,11567.164179,Petrol


In [9]:
log("Transform phase Started")
transformed_data = transform(extracted_data)
log("Transform phase Ended")
transformed_data

,car_model,year_of_manufacture,price,fuel
0,baleno,2016,8731.34,Petrol
1,alto k10,2014,3805.97,Petrol
2,sx4,2008,2910.45,Petrol
3,dzire,2014,8208.96,Diesel
4,omni,2012,1865.67,Petrol
5,ciaz,2014,11194.03,Petrol
6,ritz,2013,3955.22,Petrol
7,wagon r,2006,1567.16,Petrol
8,ertiga,2015,8656.72,Petrol
9,ciaz,2017,11567.16,Petrol


In [10]:
log("Load phase Started")
load(targetfile,transformed_data)
log("Load phase Ended")

In [11]:
log("ETL Job Ended")